<h1>Wirecell Clustering v0_30_2</h1>

<h6>
<u>File</u>: Clustering v0_30_2.ipynb <br>
<br>
<u>Author</u>: Ewerton Belchior <br>
<br>
<u>Goal</u>: General information on how to setup sbndcode and wirecell for clustering evaluation. <br>
<br>

<h5>Setup sbndcode

In [ ]:
/cvmfs/oasis.opensciencegrid.org/mis/apptainer/current/bin/apptainer shell --shell=/bin/bash -B /cvmfs,/exp,/nashome,/pnfs/sbn,/pnfs/sbnd,/opt,/run/user,/etc/hostname,/etc/hosts,/etc/krb5.conf --ipc --pid /cvmfs/singularity.opensciencegrid.org/fermilab/fnal-dev-sl7:latest

In [ ]:
source /cvmfs/sbnd.opensciencegrid.org/products/sbnd/setup_sbnd.sh
setup sbndcode v10_04_07 -q e26:prof
cd /exp/sbnd/app/users/ebatista/wirecell-clustering/work

<h5>Buil wct for debugging only (no need to build/export LD if no debugging)

In [ ]:
cd /exp/sbnd/app/users/ebatista/wirecell-clustering/work/wire-cell-toolkit_v0.30.2
./wcb configure \
--prefix=/exp/sbnd/app/users/ebatista/wirecell-clustering/work/wire-cell-toolkit_v0.30.2/install \
--with-spdlog=/cvmfs/larsoft.opensciencegrid.org/products/spdlog/v1_9_2 \
--with-spdlog-include=/cvmfs/larsoft.opensciencegrid.org/products/spdlog/v1_9_2/Linux64bit+3.10-2.17-e26-prof/include \
--with-spdlog-lib=/cvmfs/larsoft.opensciencegrid.org/products/spdlog/v1_9_2/Linux64bit+3.10-2.17-e26-prof/lib64 \
--with-jsoncpp=/cvmfs/larsoft.opensciencegrid.org/products/jsoncpp/v1_9_5a \
--with-jsoncpp-include=/cvmfs/larsoft.opensciencegrid.org/products/jsoncpp/v1_9_5a/Linux64bit+3.10-2.17-e26-prof/include \
--with-jsoncpp-lib=/cvmfs/larsoft.opensciencegrid.org/products/jsoncpp/v1_9_5a/Linux64bit+3.10-2.17-e26-prof/lib \
--with-eigen=/cvmfs/larsoft.opensciencegrid.org/products/eigen/v23_08_01_66e8f \
--with-eigen-include=/cvmfs/larsoft.opensciencegrid.org/products/eigen/v23_08_01_66e8f/include/eigen3 \
--with-jsonnet-lib=/cvmfs/larsoft.opensciencegrid.org/products/jsonnet/v0_17_0a/Linux64bit+3.10-2.17-e20-prof/lib \
--with-jsonnet-include=/cvmfs/larsoft.opensciencegrid.org/products/jsonnet/v0_17_0a/Linux64bit+3.10-2.17-e20-prof/include \
--boost-includes=/cvmfs/larsoft.opensciencegrid.org/products/boost/v1_82_0/Linux64bit+3.10-2.17-e26-prof/include \
--boost-libs=/cvmfs/larsoft.opensciencegrid.org/products/boost/v1_82_0/Linux64bit+3.10-2.17-e26-prof/lib
./wcb -p -v --notests install
cd -

In [ ]:
export LD_LIBRARY_PATH=/exp/sbnd/app/users/ebatista/wirecell-clustering/work/wire-cell-toolkit_v0.30.2/install/lib:$LD_LIBRARY_PATH

<h5>Buil larreco for truthDepos clustering using CellTree_Module.cc (clone just once. Build again when needed)

In [ ]:
cd /exp/sbnd/app/users/ebatista/wirecell-clustering/work
mkdir larsoft_v10_04_07
cd larsoft_v10_04_07
mrb newDev
source localProducts_larsoft_v10_04_07_e26_prof/setup
cd srcs
git clone -b v10_01_09 https://github.com/LArSoft/larreco.git
mrb uc
cd $MRB_BUILDDIR
mrbsetenv
mrb i --generator ninja
mrbslp
cd -

<h5>Setup larreco

In [ ]:
# Use local larreco only to cluster thruthDepos based on their timming, using CellTree_module. If not, no need to setup local products.
source larsoft_v10_04_07/localProducts_larsoft_v10_04_07_e26_prof/setup
mrbslp

In [ ]:
# example running
lar -n1 -c celltree_sbnd_ew.fcl /exp/sbnd/data/users/ebatista/wirecell-clustering/out/enuelastic/g4.root

<h5>Setup clustering  (clone just once)

In [ ]:
#git clone -b 0.30.2 https://github.com/WireCell/wire-cell-toolkit.git wire-cell-toolkit_v0.30.2

In [ ]:
export WIRECELL_PATH=/exp/sbnd/app/users/ebatista/wirecell-clustering/work/wire-cell-toolkit_v0.30.2/cfg:$WIRECELL_PATH
#export WIRECELL_PATH=/exp/sbnd/app/users/yuhw/wire-cell-data:$WIRECELL_PATH
export WIRECELL_PATH=/exp/sbnd/app/users/ebatista/wirecell-clustering/work/wire-cell-data:$WIRECELL_PATH

In [ ]:
# example running
lar -n1 --nskip 10 -c wcp-porting-img/sbnd/wcls-img-clus-data.fcl /exp/sbnd/data/users/ebatista/wirecell-clustering/eval/run18005/sigproc.root
python wcp-porting-img/sbnd/merge-zip.py
source wcp-porting-img/upload-to-bee.sh mabc.zip

<h4>Script to generate samples, run clustering and save output in txt</h4>

([Click here to open create_cluster_samples.sh](http://localhost:8888/notebooks/OneDrive/Documentos/backup-2025-xx-xx/Meetings/WireCell/Clustering/wirecell-clustering/create_cluster_samples.sh?))

In [ ]:
cd /exp/sbnd/app/users/ebatista/wirecell-clustering/work
bash create_cluster_samples.sh